In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from matplotlib.gridspec import GridSpec

from matplotlib.ticker import MultipleLocator, StrMethodFormatter
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec

from scipy.stats import gaussian_kde, chi2, norm

%matplotlib widget

In [2]:
def sample_param(mean, err_l, err_u, n=50000, allow_negative=False, rng=None):
    rng = np.random.default_rng(rng)
    samples = np.where(
        rng.random(n) < 0.5,
        rng.normal(mean, err_l, size=n),
        rng.normal(mean, err_u, size=n)
    )
    if not allow_negative:
        samples = np.clip(samples, 0, None)
    return samples

def build_samples(df, param_names, n=50000, rng=None):
    samples = {}
    for p in param_names:
        allow_negative = (p == 'texp')  # only texp may go negative
        samples[p] = sample_param(df[p].values[0], 
                                  df[p + '_l'].values[0],
                                  df[p + '_u'].values[0],
                                  n=n, allow_negative=allow_negative, rng=rng)
    return samples


In [17]:
def tension_from_two_posteriors(
    samples1, samples2, weights1=None, weights2=None,
    n_pairs=50000, bandwidth='scott', rng=None):
    """
    Compute tension between two 2-D posteriors using HPD mass of delta=0.

    samples1, samples2 : arrays (N,2) and (M,2) of (H,m) samples
    weights1, weights2 : optional 1-D weights (nonnegative), length N and M
    n_pairs            : number of independent (i,j) pairs to draw for deltas
    bandwidth         : KDE bw for gaussian_kde ('scott', 'silverman', or float)
    rng               : np.random.Generator for reproducibility

    Returns dict with:
      alpha, n_sigma_2d, n_sigma_1d,
      density_at_zero, map_delta, density_map_delta
      alpha_gauss, n_sigma_2d_gauss, n_sigma_1d_gauss (Gaussian quick check)
    """
    rng = np.random.default_rng() if rng is None else rng
    s1 = np.asarray(samples1); s2 = np.asarray(samples2)
    assert s1.shape[1]==2 and s2.shape[1]==2, "Use 2-D samples."

    N, M = len(s1), len(s2)

    # Normalize weights if given
    if weights1 is not None:
        w1 = np.asarray(weights1, float); w1 = np.clip(w1, 0, np.inf)
        w1 /= w1.sum()
    else:
        w1 = None
    if weights2 is not None:
        w2 = np.asarray(weights2, float); w2 = np.clip(w2, 0, np.inf)
        w2 /= w2.sum()
    else:
        w2 = None

    # Draw independent indices for the convolution via random pairing
    i = rng.integers(0, N, size=n_pairs)
    j = rng.integers(0, M, size=n_pairs)

    deltas = s1[i] - s2[j]                       # shape (n_pairs, 2)
    deltas_T = deltas.T                          # (2, n_pairs)

    # Pair weights = product of marginals for independent draws
    if (w1 is None) and (w2 is None):
        w = None
    else:
        wi = (w1[i] if w1 is not None else np.full(n_pairs, 1.0/N))
        wj = (w2[j] if w2 is not None else np.full(n_pairs, 1.0/M))
        w  = wi * wj
        w /= w.sum()

    # KDE on deltas
    kde = gaussian_kde(deltas_T, weights=w, bw_method=bandwidth)
    dens_deltas = kde.evaluate(deltas_T)         # (n_pairs,)
    dens_zero   = float(kde.evaluate(np.zeros((2,1)))[0])  # at delta=(0,0)

    # MAP delta (mode among the sampled deltas)
    imap = np.argmax(dens_deltas)
    map_delta = deltas[imap]
    dens_map  = float(dens_deltas[imap])

    # HPD mass through zero: α = P(f >= f(0))
    if w is None:
        alpha = np.mean(dens_deltas >= dens_zero)
    else:
        alpha = float(w[dens_deltas >= dens_zero].sum())

    # Convert to sigma
    n_sigma_2d = np.sqrt(chi2.ppf(alpha, df=2))
    n_sigma_1d = norm.ppf((1 + alpha) / 2.0)

    # -------- Gaussian quick-check (for reference) --------
    mu1 = (s1 if w1 is None else (s1 * w1[:,None]).sum(axis=0)) if w1 is not None else s1.mean(axis=0)
    mu2 = (s2 if w2 is None else (s2 * w2[:,None]).sum(axis=0)) if w2 is not None else s2.mean(axis=0)
    C1  = np.cov(s1.T, aweights=(w1 if w1 is not None else None), bias=False)
    C2  = np.cov(s2.T, aweights=(w2 if w2 is not None else None), bias=False)
    dmu = mu1 - mu2
    Csum = C1 + C2
    # Regularize in case of near-singularity
    eps = 1e-12 * np.trace(Csum)
    Csum_reg = Csum + eps * np.eye(2)
    T2 = float(dmu @ np.linalg.solve(Csum_reg, dmu))  # Δχ² under Gaussian approx
    alpha_g = float(chi2.cdf(T2, df=2))
    n_sigma_2d_g = np.sqrt(T2)
    n_sigma_1d_g = norm.ppf((1 + alpha_g) / 2.0)

    return dict(
        alpha=alpha, n_sigma_2d=n_sigma_2d, n_sigma_1d=n_sigma_1d,
        density_at_zero=dens_zero, map_delta=map_delta, density_map_delta=dens_map,
        alpha_gauss=alpha_g, n_sigma_2d_gauss=n_sigma_2d_g, n_sigma_1d_gauss=n_sigma_1d_g
    )
    
def compute_tension_for_param(samples1, samples2, mode='1d'):
    if mode == '1d':
        # Simple Gaussian sigma comparison
        mean1, mean2 = np.mean(samples1), np.mean(samples2)
        std1, std2 = np.std(samples1), np.std(samples2)
        delta = abs(mean1 - mean2)
        sigma = np.sqrt(std1**2 + std2**2)
        return delta / sigma  # n_sigma_1d
    else:
        # Convert to 2D and use full KDE method
        S1 = np.column_stack([samples1[:,0], samples1[:,1]])
        S2 = np.column_stack([samples2[:,0], samples2[:,1]])
        return tension_from_two_posteriors(S1, S2)['n_sigma_2d']
    
# Example values with asymmetric uncertainties:

def compute_asymmetric_tension(a, a_err_plus, a_err_minus,
                                b, b_err_plus, b_err_minus):

    if a > b:
        sigma_eff = np.sqrt(a_err_minus**2 + b_err_plus**2)
    else:
        sigma_eff = np.sqrt(a_err_plus**2 + b_err_minus**2)

    # Sigma tension
    delta = abs(a - b)
    tension_sigma = delta / sigma_eff
    return tension_sigma

In [4]:
columns = ['Mej', 'Mej_u', 'Mej_l', 
           'vej', 'vej_u', 'vej_l', 
           'Tmin', 'Tmin_u', 'Tmin_l',
           'texp', 'texp_u', 'texp_l',
           'Mscm', 'Mscm_u', 'Mscm_l',
           's', 's_u', 's_l',
           'rho0', 'rho0_u', 'rho0_l',
           'R0', 'R0_u', 'R0_l',
           'n', 'n_u', 'n_l',
           'N_H', 'N_H_u', 'N_H_l',
           'sigma', 'sigma_u', 'sigma_l']


vxm_list = [38.81, 6.55, 6.02, 6918, 161, 157, 6310, 147, 144,
            -0.86, 0.08, 0.09, 1.48, 0.14, 0.13, 1.40, 0.08, 0.08,
            3.02, 6.53, 1.92, 12.3, 14, 6.9, 7.46, 0.47, 0.29, 
            5.57, 0.70, 0.63, 0.126, 0.006, 0.008]

rv_list = [20.1, 19, 14.9, 4721, 3750, 2022, 2587, 4960, 2570, 
           -10.4, 5.7, 7.7, 1.26, 6.68, 0.92, 1.37, 0.67, 0.93,
           6.31, 0.44, 5.81, 13.1, 35.2, 9.7, 9.44, 1.79, 1.80,
           0.08, 6.23, 0.08, 0.100, 0.151, 0.005]

vxm_params_df = pd.DataFrame([vxm_list], columns = columns)
rv_params_df = pd.DataFrame([rv_list], columns = columns)

param_names = ['Mej','vej','Tmin','texp','Mscm','s','rho0','R0','n','N_H','sigma']

samples_vxm = build_samples(vxm_params_df, param_names)
samples_rv  = build_samples(rv_params_df, param_names)



In [18]:
tension_results = {}

for p in param_names:
    a = vxm_params_df[p].values[0]
    a_err_plus = vxm_params_df[p + '_u'].values[0]
    a_err_minus = vxm_params_df[p + '_l'].values[0]
    b = rv_params_df[p].values[0]
    b_err_plus = rv_params_df[p + '_u'].values[0]
    b_err_minus = rv_params_df[p + '_l'].values[0]
    
    tension_results[p] = compute_asymmetric_tension(a, a_err_plus, a_err_minus, b, b_err_plus, b_err_minus)
    

In [19]:
for t in tension_results:
    print(f"{t}: {tension_results[t]:.2f} sigma")

Mej: 0.94 sigma
vej: 0.59 sigma
Tmin: 0.75 sigma
texp: 1.67 sigma
Mscm: 0.03 sigma
s: 0.04 sigma
rho0: 0.38 sigma
R0: 0.05 sigma
n: 1.06 sigma
N_H: 0.88 sigma
sigma: 0.17 sigma


In [ ]:
# Mej: 0.35 sigma
# vej: 0.23 sigma
# Tmin: 0.38 sigma
# texp: 0.68 sigma
# Mscm: 0.16 sigma
# s: 0.01 sigma
# rho0: 0.27 sigma
# R0: 0.12 sigma
# n: 0.16 sigma
# N_H: 0.60 sigma
# sigma: 0.07 sigma

In [11]:
X1 = np.column_stack([samples_vxm[p] for p in param_names])  # shape (N, k)
X2 = np.column_stack([samples_rv[p] for p in param_names])   # shape (N, k)

# Means and covariances
mu1 = X1.mean(axis=0)
mu2 = X2.mean(axis=0)
dmu = mu1 - mu2

# Use rowvar=False for cov with shape (k,k)
C1 = np.cov(X1, rowvar=False, bias=False)
C2 = np.cov(X2, rowvar=False, bias=False)
Csum = C1 + C2

# Regularize if near-singular (tiny ridge)
k = Csum.shape[0]
eps = 1e-12 * np.trace(Csum)   # scale of regularization; adjust if you need
Csum_reg = Csum + eps * np.eye(k)

# Compute T^2 (Mahalanobis / chi-square)
T2 = float(dmu @ np.linalg.solve(Csum_reg, dmu))

# p-value for chi-square with k degrees of freedom (upper-tail)
p_value = 1.0 - chi2.cdf(T2, df=k)

# Convert p-value to equivalent Gaussian sigma:
# - two-sided equivalent sigma (common to report): use isf(p/2)
# - one-sided equivalent sigma: use isf(p)
sigma_equiv_two_sided = norm.isf(p_value / 2.0)
sigma_equiv_one_sided = norm.isf(p_value)

# For quick numeric context, also show sqrt(T2) (geometric norm)
sqrt_T2 = np.sqrt(T2)

print(f"T2           = {T2:.6g}")
print(f"sqrt(T2)     = {sqrt_T2:.6g}    (geometric norm; not a p-value unless k=1)")
print(f"df (k)       = {k}")
print(f"p-value      = {p_value:.6g}")
print(f"equiv sigma (2-sided) = {sigma_equiv_two_sided:.4g}")
print(f"equiv sigma (1-sided) = {sigma_equiv_one_sided:.4g}")


T2           = 8.39303
sqrt(T2)     = 2.89707    (geometric norm; not a p-value unless k=1)
df (k)       = 11
p-value      = 0.677724
equiv sigma (2-sided) = 0.4156
equiv sigma (1-sided) = -0.4613


In [8]:
n_sigma_multi

2.8970845181652622

In [9]:
import numpy as np
from scipy.stats import chi2, norm

# samples_vxm, samples_rv are dicts of shape (n_samples,) per parameter

# 1) compute marginal means, stds, and marginal z-scores
means1 = np.array([samples_vxm[p].mean() for p in param_names])
means2 = np.array([samples_rv[p].mean() for p in param_names])
std1   = np.array([samples_vxm[p].std(ddof=1) for p in param_names])
std2   = np.array([samples_rv[p].std(ddof=1) for p in param_names])

dmu = means1 - means2
marginal_z = np.abs(dmu) / np.sqrt(std1**2 + std2**2)  # vector of n_i

print("max marginal z:", marginal_z.max())
print("all marginal z:", marginal_z)

# 2) independent-sum bound
T2_indep = np.sum(marginal_z**2)
print("T2_indep =", T2_indep, " -> n_multi_indep =", np.sqrt(T2_indep))

# 3) full multivariate Mahalanobis
X1 = np.column_stack([samples_vxm[p] for p in param_names])  # shape (N, k)
X2 = np.column_stack([samples_rv[p] for p in param_names])

mu1 = X1.mean(axis=0)
mu2 = X2.mean(axis=0)
C1 = np.cov(X1, rowvar=False, bias=False)
C2 = np.cov(X2, rowvar=False, bias=False)
Csum = C1 + C2
# regularize slightly if needed
eps = 1e-12 * np.trace(Csum)
Csum_reg = Csum + eps * np.eye(Csum.shape[0])

T2 = float(dmu @ np.linalg.solve(Csum_reg, dmu))
n_multi = np.sqrt(T2)

pval = 1.0 - chi2.cdf(T2, df=len(param_names))
# convert to equivalent two-sided Gaussian sigma
n_equiv_1d = norm.isf(pval/2.0)

print("T2 full =", T2, " -> n_multi =", n_multi)
print("p-value (chi2, df=k) =", pval)
print("Equivalent 1D two-sided sigma =", n_equiv_1d)


max marginal z: 1.443367911766139
all marginal z: [1.06807594 0.74962898 1.02414233 1.4156085  0.25551459 0.02083806
 0.53739927 0.20533953 1.07485029 1.44336791 0.16144196]
T2_indep = 8.41690655081964  -> n_multi_indep = 2.9011905402471654
T2 full = 8.3930332714797  -> n_multi = 2.8970732250807365
p-value (chi2, df=k) = 0.6777236564445563
Equivalent 1D two-sided sigma = 0.4155714025031945


In [10]:
n_multi

2.8970732250807365

In [24]:
tension_values = []

for p in param_names:
    tension_values.append(tension_results[p])
    
tension_values = np.array(tension_values)

In [25]:
k = len(tension_values)

chi2_obs = np.nansum(tension_values**2)
p_chi2 = 1 - chi2.cdf(chi2_obs, df=k)      # tail prob
sigma_eq = norm.isf(p_chi2/2)              # two-sided -> equivalent sigma

print("chi2 =", chi2_obs, " df =", k)
print("global p =", p_chi2)
print("equivalent sigma =", sigma_eq)

chi2 = 6.665304022450252  df = 11
global p = 0.8255042249785753
equivalent sigma = 0.22047116745532652


In [55]:
np.sqrt(chi2_obs/11)

0.7784193788143119

In [50]:

au_to_cm = 1.495978707e13  # cm in one AU
solar_mass_to_grams = 1.98847e33  # grams in one solar mass

mcsm_samples = sample_param(1.48, 0.13, 0.14, n=50000, allow_negative=False, rng=None)
rho0_samples = sample_param(3.02e-12, 1.92e-12, 6.53e-12, n=50000, allow_negative=False, rng=None)
r0_samples = sample_param(12.3, 6.9, 14, n=50000, allow_negative=False, rng=None)
s_samples = sample_param(1.40, 0.08, 0.08, n=50000, allow_negative=False, rng=None)



In [51]:
denom = (4 * np.pi * rho0_samples * (r0_samples * au_to_cm)**(s_samples)) 
denom[denom == 0] = 1e9  # avoid division by zero

rcsm = ((3*mcsm_samples*solar_mass_to_grams)/+ r0_samples**3)**(1/3)

/var/folders/6v/pg_c9mqn3gn45gn_zmvy9w9cvfsx9j/T/ipykernel_11100/3524349800.py:4: RuntimeWarning: divide by zero encountered in divide
  rcsm = ((3*mcsm_samples*solar_mass_to_grams)/+ r0_samples**3)**(1/3)


In [52]:
denom

array([1.00000000e+09, 1.00000000e+09, 6.39615690e+08, ...,
       1.79675309e+09, 1.89057208e+11, 6.16915090e+10])

In [53]:
np.nanpercentile(rcsm, 50)/au_to_cm, (np.nanpercentile(rcsm, 50) - np.nanpercentile(rcsm, 16))/au_to_cm, (np.nanpercentile(rcsm, 84) - np.nanpercentile(rcsm, 50))/au_to_cm

(0.0011210642992160456, 0.0004977869896140504, 0.004237792638545911)

In [28]:
solar_mass_to_grams = 1.98847e33  # grams in one solar mass

epsilon = 0.5

Eej = epsilon**(-2/(n_samples - 3)) * (solar_mass_to_grams * mej_samples)**((n_samples-5)/(n_samples-3))

In [30]:
np.nanpercentile(Eej, 50), np.nanpercentile(Eej, 50) - np.nanpercentile(Eej, 16), np.nanpercentile(Eej, 84) - np.nanpercentile(Eej, 50)

(2.320023445431379e+19, 2.2268655759949582e+19, 3.3320022095361534e+20)